## Task
Given a collection of image where a few of them have been labelled with known classes, find novel classes for images that are not labelled.
* Knowing the name of the new class without any other inormation is impossible.
* What can be done is the rest of the images of the dataset can be clustered based on how close they are to the known classes.

**Main Ideas:**
* The common approach of bootstrapping an image representation using the labeled data only introduces an unwanted bias, and that this can be avoided by using self-supervised learning to train the representation from scratch on the union of labelled and unlabelled data.
* Use rank statistics to transfer the model’s knowledge of the labelled classes to the problem of clustering the unlabelled images
*  Train the data representation by optimizing a joint objective function on the labelled and unlabelled subsets of the data, improving both the supervised classification of the labelled data, and the clustering of the unlabelled data

**STEP 1**

* The first idea is to pre-train the image representation (a CNN) using all available images, both labelled and 
unlabelled, using a self-supervised learning objective. Crucially, this objective doesnot leverage the known 
labels, resulting in features that are much less biased towards the labelled classes.

* Labels are used only after pre-training to learn a classifier specific to the labelled data as well as to fine-tune the deepest layers of the CNN, for which self-supervision is not as effective (A critical analysis of self-supervision, or what we can learn from a single image).

**STEP 2**
* The second idea is a new approach to transfer the information contained in the labelled images to the problem of clustering the unlabelled ones. Information is transferred by sharing the same representation between labelled and unlabelled images, motivated by the fact that the new classes are often similar to the known ones. 

* Pairs of unlabelled images are compared via their representation vectors. The comparison is done using robust rank statistics, by testing if two images share the same subset of k maximally activated representation components. This test is used to decide if two unlabelled images belong to the same (new) class or not, generating a set of noisy pairwise pseudo-labels. The pseudo-labels are then used to learn a similarity function for the unlabelled images.

**STEP 3**
* After bootstrapping the representation, to optimise the model by minimizing a joint objective function, containing terms for both the labelled and unlabelled subsets, using respectively the given labels and the generated pseudo-labels, thus avoiding the forgetting issue that may arise with a sequential approach.

* A further boost is obtained by incorporating incremental learning of the discovered classes in the classification task, which allows information to flow between the labelled and unlabelled images.

**Ranking Statistics**

Rather than comparing vectors $z_{i}^{u}$, $z_{j}^{u}$ directly (e.g., by a scalar product), however, we use a more
robust rank statistics. Specifically, we rank the values in vector $z_{i}^{u}$ by magnitude. Then, if the rankings obtained for two unlabelled images $x_{i}^{u}$ and $x_{j}^{u}$ are the same, they are very likely to belong to the same (new) class, so we set $s_{ij} = 1$. Otherwise, we set $s_{ij}$ = 0. In practice, it is too strict to require the two rankings to be identical if the dimension of $z_{i}^{u}$ is high (otherwise we may end up with $s_{ij}$ = 0 for all pairs $(i, j), i \neq j$). Therefore, we relax this requirement by only testing if the sets of
the top-k ranked dimensions are the same (we use k = 5 in our experiments)

The "moving target" phenomenon associated with the ranking statistics of the images is regularized by adding a consistency loss that penalizes the model if transformations of the same image should result in different outcomes for both labelled and unlabelled data.

**Overall Loss:** $L = L_{CE} + L_{BCE} + ω(t) * L_{MSE}$

$L_{CE} = - 1/N \sum_{i=1}^{N}log(\eta_{y_i}^l(z_{i}^{l}))$ 

$L_{BCE} = - 1/M^2 \sum_{i=1}^{M} \sum_{j=1}^{M} [s_{ij} log(\eta^{u} (z_{i}^{u})^T \eta^{u}(z_{j}^{u})) + (1 - s_{ij}) log(1 - \eta^{u} (z_{i}^{u})^T \eta^{u}(z_{j}^{u}))]$

$L_{MSE} = 1/N \sum_{i=1}^{N}(\eta^l(z_{i}^{l}) - \eta^l(\hat{z}_{i}^{l}))^2 + 1/M \sum_{i=1}^{M}(\eta^u(z_{i}^{u}) - \eta^u(\hat{z}_{i}^{u}))^2$

$ω(t) = \lambda \exp^{-5(1 - t/T)^2}$